# Train 2D landmarks detector

In [ ]:
import os
import matplotlib
import cv2
import sys
import numpy as np
from matplotlib import pyplot as plt
from datatools import Loader2D, Generator, Aug2D
from traintools import plot_history, get_callbacks, create_dirs
from testtools import plot2D

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # gpu id to use

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Main settings

In [ ]:
data_dir = "300W"
project_name = "new"
imsize = (64, 64)
used_lmarks = None

logdir = create_dirs(project_name)

In [ ]:
# Choise only needed landmarks or skip this cell
used_lmarks = []
for i in [8, 30, 36, 39, 42, 45, 48, 54]:
    used_lmarks += list(range(i*2, i*2+2))

## Data loaders initialization and checkup

In [ ]:
# Init loader for training images
loader_train = Loader2D(data_dir=os.path.join(data_dir, "train_crop"), 
                        img_size=imsize, 
                        valid_size=0, 
                        augmenter=Aug2D(),
                        used_lmarks=used_lmarks
                       )   
# Init loader for test/validation images
loader_valid = Loader2D(data_dir=os.path.join(data_dir, "test_crop"), 
                        img_size=imsize, 
                        valid_size=1, 
                        augmenter=None,
                        used_lmarks=used_lmarks
                       )  

In [ ]:
# Test a generator
n = 0
for x, y in Generator(loader_valid, 8).get_iterator(train=False):
    n += 1
    print(x.shape, y.shape)
    if n >= 3:
        break

In [ ]:
x, y = loader_train.get_item(loader_train.train_set[np.random.randint(10)])
plot2D(x[..., 0], y.reshape(-1, 2))
print("image min/max:", x.min(), x.max())

## Model initialization

In [ ]:
from models import fconv2d as lmarks_model
from tensorflow.keras.optimizers import Adam

model = lmarks_model((imsize[1], imsize[0], 1), lmarks_count=8, normalize_input=1)
model.compile(loss="mean_absolute_error", optimizer=Adam())
print(model.summary(100))

## Start training

In [ ]:
gen_train = Generator(loader_train, 128)
gen_valid = Generator(loader_valid, 128)

history = model.fit_generator(generator=gen_train.get_iterator(train=True),
                              steps_per_epoch=20,
                              epochs=400,
                              initial_epoch=0,
                              validation_data=gen_valid.get_iterator(train=False),
                              validation_steps=25,
                              callbacks=get_callbacks(logdir, 0.002, 350),
                              use_multiprocessing=False
                              )

model.save(os.path.join(logdir, "checkpoints", "model_final.h5"))
plot_history(logdir, history)

## Model validation

In [ ]:
# Choise model to test
from tensorflow.keras.models import load_model
model = load_model(os.path.join("vanilla-68", "checkpoints", "model_final.h5"))
model.compile(loss="mean_absolute_error")

In [ ]:
# CALC TEST METRICS WITHOUT AUGMENTATION
bsize = 5
gen_test = Generator(loader_valid, bsize)
model.evaluate(gen_test.get_iterator(train=False), steps=len(loader_valid.valid_set)//bsize)

In [ ]:
item_id = np.random.randint(10)
x, y = loader_valid.get_item(loader_valid.valid_set[item_id])

lmarks = model.predict(np.expand_dims(x, 0)).flatten()
print(model.evaluate(np.expand_dims(x, 0), np.expand_dims(y, 0), steps=1, verbose=0))
plot2D(x[..., 0], y.reshape(-1, 2), lmarks.reshape(-1, 2));

In [ ]:
from testtools import CropTester, keras_wrapper

ct = CropTester(keras_wrapper(model), num_iterations=5, pad_min_max=(0.0, 0.2), crop_type="rand", rseed=1)
ct.test_image("300W/test_crop/helen/000000.png", color=(255, 0, 0))
ct.scatter_all()
ct.mean_pixel_std()

## Freeze model

In [ ]:
model = lmarks_model((imsize[1], imsize[0], 1))
model.load_weights(os.path.join(logdir, "checkpoints/model_final.h5"), by_name=True)
model.save(os.path.join(logdir, "checkpoints/model_infer.h5"))

In [ ]:
import subprocess
import sys
subprocess.Popen([r"/bin/bash"])
process_params = [r"{}".format(sys.executable),
                  r"freeze_graph.py",
                  r"{}".format(os.path.join(logdir, "checkpoints/model_infer.h5"))]
res = subprocess.Popen(process_params)

# Is equal to:
# !/home/rybin/miniconda3/envs/dm/bin/python freeze_graph.py vanilla-68/checkpoints/model_infer.h5